# Purpose

Build out 1+ simple RAG pipelines to see how well we can try and troubleshoot issues via a chat interface as a starting point (and using a few different manuals we found online for DCFC chargers as the corpus).

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from rich import print
import os

# Constants

In [2]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

# Check Available Models

In [3]:
from evlens.models.openai_tools import find_models

In [4]:
find_models('gpt-3')

['gpt-3.5-turbo-0301',
 'gpt-3.5-turbo-1106',
 'gpt-3.5-turbo-16k-0613',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo-instruct',
 'gpt-3.5-turbo-instruct-0914']

# llama-index Tutorials

We will go ahead and see how to use `llama-index` for a quick RAG implementation to simply POC things and see what we can do. 

## [Quickstart](https://docs.llamaindex.ai/en/stable/getting_started/starter_example.html)

In [10]:
%%time
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("../data/pg/").load_data()
index = VectorStoreIndex.from_documents(documents)

DEBUG:llama_index.readers.file.base:> [SimpleDirectoryReader] Total files added: 1
> [SimpleDirectoryReader] Total files added: 1
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: What I Worked On

February 2021

Before college...
> Adding chunk: What I Worked On

February 2021

Before college...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: I couldn't have put this into words when I was ...
> Adding chunk: I couldn't have put this into words when I was ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: So I looked around to see what I could salvage ...
> Adding chunk: So I looked around to see what I could salvage ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: I didn't want to drop out of grad school, but h...
> Adding chunk: I didn't want to drop out of grad school, but h...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: We actually had one of those little stoves, fed...
> Adding chunk: We actually had one of those little stoves, fe

In [9]:
%%time
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)

Response(
    response='The author worked on writing and programming outside of school before college. They wrote short 
stories and tried writing programs on an IBM 1401 computer. They later got a microcomputer and started programming 
more extensively, writing simple games and a word processor.',
    source_nodes=[
        NodeWithScore(
            node=TextNode(
                id_='e636bb44-f684-451a-828d-10fe6334c952',
                embedding=None,
                metadata={
                    'file_path': '../data/pg/paul_graham_essay.txt',
                    'file_name': 'paul_graham_essay.txt',
                    'file_type': 'text/plain',
                    'file_size': 75042,
                    'creation_date': '2024-01-21',
                    'last_modified_date': '2024-01-21',
                    'last_accessed_date': '2024-01-21'
                },
                excluded_embed_metadata_keys=[
                    'file_name',
                    'file_type',
                    'file_size',
                    'creation_date',
                    'last_modified_date',
                    'last_accessed_date'
                ],
                excluded_llm_metadata_keys=[
                    'file_name',
                    'file_type',
                    'file_size',
                    'creation_date',
                    'last_modified_date',
                    'last_accessed_date'
                ],
                relationships={
                    <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
                        node_id='de73c3b9-df56-458f-aa09-34e5748e54a9',
                        node_type=<ObjectType.DOCUMENT: '4'>,
                        metadata={
                            'file_path': '../data/pg/paul_graham_essay.txt',
                            'file_name': 'paul_graham_essay.txt',
                            'file_type': 'text/plain',
                            'file_size': 75042,
                            'creation_date': '2024-01-21',
                            'last_modified_date': '2024-01-21',
                            'last_accessed_date': '2024-01-21'
                        },
                        hash='0ecab27d48a07c4b35aeadf5fd617f422f81d20b8891f0de99a5c05c30410dde'
                    ),
                    <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
                        node_id='b796f7df-f150-4c25-82ee-199184296b1b',
                        node_type=<ObjectType.TEXT: '1'>,
                        metadata={},
                        hash='692d74391b6ad961556d19669374a543e4752bab10e415d8eeeda9ccf682a597'
                    )
                },
                hash='57bbc3959d606ec9694eef722d1b8af2fe66ed941ab6183f90aa2b3c0a109751',
                text='What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside 
of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to 
write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters
with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 
that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14.
The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and
I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking 
machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent 
lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, then 
stack them in the card reader and press a button to load the program into memory and run it. The result would 
ordinarily be to print something on the spectacularly loud printer.\n\nI was puzzled by the 1401. I couldn\'

CPU times: user 80.4 ms, sys: 5.44 ms, total: 85.8 ms
Wall time: 3.66 s


In [11]:
# Save embeddings from in-memory vector store to disk for later usage
# Saves to a directory storage/ in same directory as where the script is housed
index.storage_context.persist(persist_dir='./storage')

DEBUG:fsspec.local:open file: /Users/davemcrench/Documents/Projects/evlens/notebooks/storage/docstore.json
open file: /Users/davemcrench/Documents/Projects/evlens/notebooks/storage/docstore.json
DEBUG:fsspec.local:open file: /Users/davemcrench/Documents/Projects/evlens/notebooks/storage/index_store.json
open file: /Users/davemcrench/Documents/Projects/evlens/notebooks/storage/index_store.json
DEBUG:fsspec.local:open file: /Users/davemcrench/Documents/Projects/evlens/notebooks/storage/graph_store.json
open file: /Users/davemcrench/Documents/Projects/evlens/notebooks/storage/graph_store.json
DEBUG:fsspec.local:open file: /Users/davemcrench/Documents/Projects/evlens/notebooks/storage/default__vector_store.json
open file: /Users/davemcrench/Documents/Projects/evlens/notebooks/storage/default__vector_store.json
DEBUG:fsspec.local:open file: /Users/davemcrench/Documents/Projects/evlens/notebooks/storage/image__vector_store.json
open file: /Users/davemcrench/Documents/Projects/evlens/notebook

In [12]:
# Do it again, but this time generate and store data/embeddings if they don't exist and load them up if they do
import os.path
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)

DEBUG:llama_index.storage.kvstore.simple_kvstore:Loading llama_index.storage.kvstore.simple_kvstore from ./storage/docstore.json.
Loading llama_index.storage.kvstore.simple_kvstore from ./storage/docstore.json.
DEBUG:fsspec.local:open file: /Users/davemcrench/Documents/Projects/evlens/notebooks/storage/docstore.json
open file: /Users/davemcrench/Documents/Projects/evlens/notebooks/storage/docstore.json
DEBUG:llama_index.storage.kvstore.simple_kvstore:Loading llama_index.storage.kvstore.simple_kvstore from ./storage/index_store.json.
Loading llama_index.storage.kvstore.simple_kvstore from ./storage/index_store.json.
DEBUG:fsspec.local:open file: /Users/davemcrench/Documents/Projects/evlens/notebooks/storage/index_store.json
open file: /Users/davemcrench/Documents/Projects/evlens/notebooks/storage/index_store.json
DEBUG:llama_index.graph_stores.simple:Loading llama_index.graph_stores.simple from ./storage/graph_store.json.
Loading llama_index.graph_stores.simple from ./storage/graph_stor

Response(
    response='The author, growing up, worked on writing and programming. They wrote short stories and also tried 
writing programs on an IBM 1401 computer. Later, they got a microcomputer and started programming more extensively,
writing simple games and a word processor. They initially planned to study philosophy in college but ended up 
switching to AI.',
    source_nodes=[
        NodeWithScore(
            node=TextNode(
                id_='4f16e725-9e0d-4004-a790-2e6ee8110e25',
                embedding=None,
                metadata={
                    'file_path': '../data/pg/paul_graham_essay.txt',
                    'file_name': 'paul_graham_essay.txt',
                    'file_type': 'text/plain',
                    'file_size': 75042,
                    'creation_date': '2024-01-21',
                    'last_modified_date': '2024-01-21',
                    'last_accessed_date': '2024-01-21'
                },
                excluded_embed_metadata_keys=[
                    'file_name',
                    'file_type',
                    'file_size',
                    'creation_date',
                    'last_modified_date',
                    'last_accessed_date'
                ],
                excluded_llm_metadata_keys=[
                    'file_name',
                    'file_type',
                    'file_size',
                    'creation_date',
                    'last_modified_date',
                    'last_accessed_date'
                ],
                relationships={
                    <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
                        node_id='94049d29-e873-4d38-a6c8-6952d0f3ec2e',
                        node_type=<ObjectType.DOCUMENT: '4'>,
                        metadata={
                            'file_path': '../data/pg/paul_graham_essay.txt',
                            'file_name': 'paul_graham_essay.txt',
                            'file_type': 'text/plain',
                            'file_size': 75042,
                            'creation_date': '2024-01-21',
                            'last_modified_date': '2024-01-21',
                            'last_accessed_date': '2024-01-21'
                        },
                        hash='0ecab27d48a07c4b35aeadf5fd617f422f81d20b8891f0de99a5c05c30410dde'
                    ),
                    <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
                        node_id='3e0ddcba-6dfd-4332-ac3e-15360ebc7fb1',
                        node_type=<ObjectType.TEXT: '1'>,
                        metadata={},
                        hash='692d74391b6ad961556d19669374a543e4752bab10e415d8eeeda9ccf682a597'
                    )
                },
                hash='57bbc3959d606ec9694eef722d1b8af2fe66ed941ab6183f90aa2b3c0a109751',
                text='What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside 
of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to 
write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters
with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 
that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14.
The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and
I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking 
machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent 
lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, then 
stack them in the card reader and press a button to load the program into memory and run it. The result would 
ordinarily be to print something on th

In [16]:
response.response

'The author, growing up, worked on writing and programming. They wrote short stories and also tried writing programs on an IBM 1401 computer. Later, they got a microcomputer and started programming more extensively, writing simple games and a word processor. They initially planned to study philosophy in college but ended up switching to AI.'

## Quickstart on our own data

In [17]:
%%time
# import logging
# import sys

# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import os.path
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "../data/vector_dbs/llamaindex_techdocs_quickstart"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("../data/external/technical_docs/").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What methods can be used to activate EV charging equipment?")
print(response)

DEBUG:llama_index.readers.file.base:> [SimpleDirectoryReader] Total files added: 1
> [SimpleDirectoryReader] Total files added: 1
> [SimpleDirectoryReader] Total files added: 1
DEBUG:httpcore.connection:close.started
close.started
close.started
DEBUG:httpcore.connection:close.complete
close.complete
close.complete
DEBUG:httpcore.connection:close.started
close.started
close.started
DEBUG:httpcore.connection:close.complete
close.complete
close.complete
DEBUG:httpcore.connection:close.started
close.started
close.started
DEBUG:httpcore.connection:close.complete
close.complete
close.complete
DEBUG:httpcore.connection:close.started
close.started
close.started
DEBUG:httpcore.connection:close.complete
close.complete
close.complete
DEBUG:httpcore.connection:close.started
close.started
close.started
DEBUG:httpcore.connection:close.complete
close.complete
close.complete
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: EVITP (EVCS) Test Review
Study online at https:...
> Adding chunk: EVIT

Response(
    response='RFID tags and swipe cards can be used to activate EV charging equipment.',
    source_nodes=[
        NodeWithScore(
            node=TextNode(
                id_='bfa2ee05-2cc1-4fc2-bc85-8af87c0be04e',
                embedding=None,
                metadata={
                    'page_label': '1',
                    'file_name': 'evitp_training_flashcards.pdf',
                    'file_path': '../data/external/technical_docs/evitp_training_flashcards.pdf',
                    'file_type': 'application/pdf',
                    'file_size': 23191,
                    'creation_date': '2024-01-22',
                    'last_modified_date': '2024-01-21',
                    'last_accessed_date': '2024-01-22'
                },
                excluded_embed_metadata_keys=[
                    'file_name',
                    'file_type',
                    'file_size',
                    'creation_date',
                    'last_modified_date',
                    'last_accessed_date'
                ],
                excluded_llm_metadata_keys=[
                    'file_name',
                    'file_type',
                    'file_size',
                    'creation_date',
                    'last_modified_date',
                    'last_accessed_date'
                ],
                relationships={
                    <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
                        node_id='074e37d5-3f05-4362-a008-b7db26b320a7',
                        node_type=<ObjectType.DOCUMENT: '4'>,
                        metadata={
                            'page_label': '1',
                            'file_name': 'evitp_training_flashcards.pdf',
                            'file_path': '../data/external/technical_docs/evitp_training_flashcards.pdf',
                            'file_type': 'application/pdf',
                            'file_size': 23191,
                            'creation_date': '2024-01-22',
                            'last_modified_date': '2024-01-21',
                            'last_accessed_date': '2024-01-22'
                        },
                        hash='ce02c9b0c6352b86d940f623a47c0593ba986d12ea395cb65a0b5a6b494391d9'
                    ),
                    <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
                        node_id='c96b66d0-9386-4e7e-b312-1fceb9586f49',
                        node_type=<ObjectType.TEXT: '1'>,
                        metadata={},
                        hash='19567bb0b882e3087622f9364b5cc8b8c5dff0d48f464b0008cb42cd626ffcf1'
                    )
                },
                hash='ce02c9b0c6352b86d940f623a47c0593ba986d12ea395cb65a0b5a6b494391d9',
                text='EVITP (EVCS) Test Review\nStudy online at https://quizlet.com/_bc181w\n1.EVSE signage may be 
____ Subject to local codes and \nregulations\n2.Name 3 alternatives to fossil fuels __1__ Compressed air\n3.Name 3
alternatives to fossil fuels __2__ Electricity\n4.Name 3 alternatives to fossil fuels __3__ Fuel Cell\n5.EV 
charging equipment can be activated by \nwhat methods ___1___RFID Tags\n6.EV charging equipment can be activated by
\nwhat methods ___2___Swipe cards\n7.Name two advantages of wireless charging \n__1__No exposed electrical 
con-\ntact points\n8.Name two advantages of wireless charging \n__2__Better aesthetic quality\n9.What article 
covers the ground and bonding \nfor electrical systemsArticle 250\n10.What safety features does electric vehicle 
sup-\nply equipment provide? __1__GFCI protection\n11.What safety features does electric vehicle sup-\nply 
equipment provide? __2__Internal fault detection\n12.What is the maximum voltage drop recom-\nmended for branch 
circuits and feeder electri-\ncal systems in the NEC? __1__3% Branch\n13.What is the maximum voltage drop 
recom-\nmended for branch circuits and feeder electri-\ncal systems in the NEC? __2__5% Feeder system\n14.The EV 
industry __1__ Auto manufactures\n1 / 5'

CPU times: user 290 ms, sys: 35.3 ms, total: 325 ms
Wall time: 2.19 s


In [18]:
response.response

'RFID tags and swipe cards can be used to activate EV charging equipment.'

In [31]:
def get_source_files(response):
    filenames = set()
    filenames.update([n.node.metadata['file_name'] for n in response.source_nodes])
    return list(filenames)

In [32]:
get_source_files(response)

['evitp_training_flashcards.pdf']

Nice! It figured out from the quiz data how to answer that simple question, I'm encouraged.

In [33]:
response = query_engine.query("How do you repair a frayed charging cable in a Veefile tritium charger?")
print(response.response)
print("\n\n")
get_source_files(response)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x2c6d044a0>, 'json_data': {'input': ['How do you repair a frayed charging cable in a Veefile tritium charger?'], 'model': <OpenAIEmbeddingModeModel.TEXT_EMBED_ADA_002: 'text-embedding-ada-002'>, 'encoding_format': 'base64'}}
Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x2c6d044a0>, 'json_data': {'input': ['How do you repair a frayed charging cable in a Veefile tritium charger?'], 'model': <OpenAIEmbeddingModeModel.TEXT_EMBED_ADA_002: 'text-embedding-ada-002'>, 'encoding_format': 'base64'}}
Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x2c6d044a0>, 'json_data': {'input': ['How do you repair a frayed charging cable in a Veefile tritium charger?'], 'mo

To repair a frayed charging cable in a Veefile Tritium charger, you can follow these steps:
1. Unplug the charger from the power source and disconnect it from the vehicle.
2. Inspect the frayed area of the charging cable to assess the extent of the damage.
3. If the fraying is minor, you can use electrical tape to wrap around the affected area, ensuring that the wires 
are properly covered and insulated.
4. If the fraying is severe or the wires are exposed, it is recommended to replace the charging cable entirely. 
Contact the manufacturer or an authorized service center to obtain a replacement cable.
5. Once you have repaired or replaced the charging cable, ensure that it is securely connected to the Veefile 
Tritium charger and the vehicle before plugging it back into the power source.

['evitp_training_flashcards.pdf']

Shoot, seems like it effectively hallucinated that response instead of saying that the tech docs it had been given didn't include this info...

# Customizing the RAG Application

> The ServiceContext is a bundle of services and configurations used across a LlamaIndex pipeline.

## Reduce chunk size

In [35]:
# Parse into smaller chunks
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(chunk_size=1000)

In [41]:
avoid_guessing_str = "Respond with 'I do not know the answer to that.' if the answer is not in the provided context. Do not guess or speculate outside of the provided context."

In [38]:
%%time

documents = SimpleDirectoryReader("../data/external/technical_docs/").load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine = index.as_query_engine()
response = query_engine.query(f"What methods can be used to activate EV charging equipment? {avoid_guessing_str}")
print(response.response)

DEBUG:llama_index.readers.file.base:> [SimpleDirectoryReader] Total files added: 1
> [SimpleDirectoryReader] Total files added: 1
> [SimpleDirectoryReader] Total files added: 1
> [SimpleDirectoryReader] Total files added: 1
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: EVITP (EVCS) Test Review
Study online at https:...
> Adding chunk: EVITP (EVCS) Test Review
Study online at https:...
> Adding chunk: EVITP (EVCS) Test Review
Study online at https:...
> Adding chunk: EVITP (EVCS) Test Review
Study online at https:...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: EVITP (EVCS) Test Review
Study online at https:...
> Adding chunk: EVITP (EVCS) Test Review
Study online at https:...
> Adding chunk: EVITP (EVCS) Test Review
Study online at https:...
> Adding chunk: EVITP (EVCS) Test Review
Study online at https:...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: EVITP (EVCS) Test Review
Study online at https:...
> Adding chunk: EVITP (EVCS) Test Review
Study onl

RFID tags and swipe cards can be used to activate EV charging equipment.

CPU times: user 191 ms, sys: 38.5 ms, total: 229 ms
Wall time: 2.15 s


In [40]:
response = query_engine.query(f"How do you repair a frayed charging cable in a Veefile tritium charger? {avoid_guessing_str}")
print(response.response)
print("\n\n")
get_source_files(response)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x2c688d9e0>, 'json_data': {'input': ["How do you repair a frayed charging cable in a Veefile tritium charger? Respond with 'I do not know the answer to that.' if the answer is not in the provided context. Do not guess or speculate outside of the provided context."], 'model': <OpenAIEmbeddingModeModel.TEXT_EMBED_ADA_002: 'text-embedding-ada-002'>, 'encoding_format': 'base64'}}
Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x2c688d9e0>, 'json_data': {'input': ["How do you repair a frayed charging cable in a Veefile tritium charger? Respond with 'I do not know the answer to that.' if the answer is not in the provided context. Do not guess or speculate outside of the provided context."], 'model': <OpenAIEmbeddingModeModel.TEXT_EMBED_ADA_002: '

I do not know the answer to that.

['evitp_training_flashcards.pdf']

In [43]:
logging.getLogger().setLevel(logging.INFO)

## Switch vector store

> *StorageContext* defines the storage backend for where the documents, embeddings, and indexes are stored.

In [44]:
%%time

import chromadb
from llama_index.vector_stores import ChromaVectorStore
from llama_index import StorageContext

chroma_client = chromadb.PersistentClient()
chroma_collection = chroma_client.create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
CPU times: user 550 ms, sys: 135 ms, total: 684 ms
Wall time: 1.73 s


In [48]:
# Whoops, wasn't searching subdirectories!
documents = SimpleDirectoryReader("../data/external/technical_docs/", recursive=True).load_data()

In [49]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)
query_engine = index.as_query_engine()
response = query_engine.query(f"How do you repair a frayed charging cable in a Veefile tritium charger? {avoid_guessing_str}")
print(response.response)
print("\n\n")
get_source_files(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 20

I do not know the answer to that.

['Users-Manual-3059907.pdf']

In [51]:
set([d.metadata['file_name'] for d in documents])

{'EFAPOWER EV-QC45 Standalone - V 05.01 (100-135).pdf',
 'External-Photos-3059906.pdf',
 'Internal-Photos-3059905.pdf',
 'README.md',
 'Users-Manual-3059907.pdf',
 'evitp_training_flashcards.pdf',
 'instruction_manual.pdf'}

## Increase Context (by increasing chunk counts retrieved)

> `as_query_engine` builds a default retriever and query engine on top of the index. You can configure the retriever and query engine by passing in keyword arguments. Here, we configure the retriever to return the top 5 most similar documents (instead of the default of 2).

In [52]:
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(similarity_top_k=5)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [53]:
response = query_engine.query(f"How do you repair a frayed charging cable in a Veefile tritium charger? {avoid_guessing_str}")
print(response.response)
print("\n\n")
get_source_files(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


I do not know the answer to that.

['Users-Manual-3059907.pdf']

## Swap Out LLMs

In [54]:
from llama_index import ServiceContext
from llama_index.llms import PaLM

service_context = ServiceContext.from_defaults(llm=PaLM())

ValueError: PaLM is not installed. Please install it with `pip install google-generativeai`.

## Changing [Response Mode](https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/response_modes.html)

This does *nothing* to the retrieval part of the process, it simply changes what the LLM does with the chunks before spitting out the final answer (e.g. instead of stuffing everything you retrieved into a single prompt and discarding anything over the token limit, it will summarize chunks through some strategy to make them fit the context window).